In [24]:
import json

def anonymize_labels(
    json_path,
    output_path="golden_dataset_with_replacements.json",
    placeholder_format="__{label}__"
):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    for entry in data:
        original_text = entry["text"]
        text_anonymized = original_text
        labels = sorted(entry["labels"], key=lambda x: x["start"], reverse=True)
        replacements = []

        seen_spans = set()

        for label in labels:
            start, end = label["start"], label["end"]
            key = (start, end, label["label"])
            if key in seen_spans:
                continue
            seen_spans.add(key)

            original_value = original_text[start:end]
            placeholder = placeholder_format.format(label=label["label"])

            # Ersetze im anonymisierten Text
            text_anonymized = text_anonymized[:start] + placeholder + text_anonymized[end:]

            replacements.append({
                "label": label["label"],
                "original": original_value,
                "placeholder": placeholder
            })

        entry["text_anonymized"] = text_anonymized
        entry["replacements"] = list(reversed(replacements))  # für nachvollziehbare Reihenfolge

    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"✅ Datei mit Original + Anonymisiertem Text gespeichert unter: {output_path}")

# Beispiel-Aufruf:
anonymize_labels("../../data/original/granular_dataset_split_norm/train_norm.json")


✅ Datei mit Original + Anonymisiertem Text gespeichert unter: golden_dataset_with_replacements.json


In [14]:
snippets = [
    "Dr.",
    "Isabelle", "Eckbauer",
    "Ahmet", "Gerlach",
    "+49 (0) 5402 008802",
    "+49(0)7195176538",
    "1ISK0070547123",
    "0022386623",
    "01.08.2023",
    "Franz-Xaver", "Huhn"
]

import unicodedata
import re

def normalize(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.replace('\xa0', ' ')
    text = re.sub(r'[\u200b\u2028\u2029\ufeff]', '', text)
    text = text.replace('\r\n', '\n').replace('\r', '\n')
    return text

text = """Guten Tag,
hiermit schicke ich Ihnen die noch fehlenden Angaben:
Übergabedatum: 01.08.2023
Mieter: Isabelle Eckbauer (+49 (0) 5402 008802)
 Ahmet Gerlach (+49(0)7195176538)
Vermieter: Dr. Franz-Xaver Huhn
Zählernummer: 1ISK0070547123
Zählerstand: 0022386623
Vielen Dank
Isabelle Eckbauer
"""

print("🔍 Originaltext:\n", text)
print("\n🧼 Normalisiert:\n", normalize(text))


🔍 Originaltext:
 Guten Tag,
hiermit schicke ich Ihnen die noch fehlenden Angaben:
Übergabedatum: 01.08.2023
Mieter: Isabelle Eckbauer (+49 (0) 5402 008802)
 Ahmet Gerlach (+49(0)7195176538)
Vermieter: Dr. Franz-Xaver Huhn
Zählernummer: 1ISK0070547123
Zählerstand: 0022386623
Vielen Dank
Isabelle Eckbauer


🧼 Normalisiert:
 Guten Tag,
hiermit schicke ich Ihnen die noch fehlenden Angaben:
Übergabedatum: 01.08.2023
Mieter: Isabelle Eckbauer (+49 (0) 5402 008802)
 Ahmet Gerlach (+49(0)7195176538)
Vermieter: Dr. Franz-Xaver Huhn
Zählernummer: 1ISK0070547123
Zählerstand: 0022386623
Vielen Dank
Isabelle Eckbauer



In [17]:
normalized = normalize(text)

for snippet in snippets:
    match = re.search(re.escape(snippet), normalized)
    if match:
        print(f"✅ Gefunden: '{snippet}' bei Position {match.start()}–{match.end()}")
    else:
        print(f"❌ NICHT gefunden: '{snippet}'")


✅ Gefunden: 'Dr.' bei Position 184–187
✅ Gefunden: 'Isabelle' bei Position 99–107
✅ Gefunden: 'Eckbauer' bei Position 108–116
✅ Gefunden: 'Ahmet' bei Position 140–145
✅ Gefunden: 'Gerlach' bei Position 146–153
✅ Gefunden: '+49 (0) 5402 008802' bei Position 118–137
✅ Gefunden: '+49(0)7195176538' bei Position 155–171
✅ Gefunden: '1ISK0070547123' bei Position 219–233
✅ Gefunden: '0022386623' bei Position 247–257
✅ Gefunden: '01.08.2023' bei Position 80–90
✅ Gefunden: 'Franz-Xaver' bei Position 188–199
✅ Gefunden: 'Huhn' bei Position 200–204
